In [9]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoStore

In [3]:
loader = TweetLoader()
loader.extract_tweets(count=5)

In [13]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Bloomberg
Boston Celtics
Bloomberg
Forbes
Forbes


In [10]:
store = MongoStore("twitter_dashboard_db", "home_timeline")

In [14]:
store.save_tweets(loader.tweets)

1. business
2. celtics
3. business
4. Forbes
5. Forbes


In [15]:
store.load_mongo_data()

[{'user': 'business',
  'tweets': [{'created_at': 'Sat Nov 26 00:36:04 +0000 2022',
    'id': 1596301729187758081,
    'id_str': '1596301729187758081',
    'text': 'On Zero, Akshat Rathi speaks to environmental journalist George Monbiot about how to radically rethink the food sys… https://t.co/Scz7hxyyNd',
    'truncated': True,
    'entities': {'hashtags': [],
     'symbols': [],
     'user_mentions': [],
     'urls': [{'url': 'https://t.co/Scz7hxyyNd',
       'expanded_url': 'https://twitter.com/i/web/status/1596301729187758081',
       'display_url': 'twitter.com/i/web/status/1…',
       'indices': [117, 140]}]},
    'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>',
    'in_reply_to_status_id': None,
    'in_reply_to_status_id_str': None,
    'in_reply_to_user_id': None,
    'in_reply_to_user_id_str': None,
    'in_reply_to_screen_name': None,
    'user': {'id': 34713362,
     'id_str': '34713362',
     'name': 'Bloomberg',
     'screen_name': 'business'